In [2]:
from Uplift_PDF import PDF
from Uplift_PDF import report_data

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300
from tqdm import tqdm
logo_path = 'logo/'
plt.rcParams['axes.unicode_minus'] = False
now = datetime.now().strftime('%Y%m%d')

day = 20240213
file_list = [i.replace('\\','/') for i in sorted(glob(f'data/{day}/process/*csv'))]
fname = [os.path.basename(i).replace('.csv','') for i in file_list]

/Users/yongseok/opt/anaconda3/envs/ML/lib/python3.11/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [3]:
report_file = [
 'data/20240213/process/n01_001.csv',
 'data/20240213/process/n02_002.csv',
 'data/20240213/process/n03_001.csv',
 'data/20240213/process/n04_021.csv',
 'data/20240213/process/n05_002.csv',
 'data/20240213/process/n06_002.csv',
 'data/20240213/process/n07_001.csv',
 'data/20240213/process/n08_003.csv',
 'data/20240213/process/n09_001.csv',
 'data/20240213/process/n10_002.csv',
 'data/20240213/process/n11_002.csv',
 'data/20240213/process/n12_001.csv',
 'data/20240213/process/n13_003.csv',
 'data/20240213/process/n14_001.csv',
 'data/20240213/process/n15_003.csv',
 'data/20240213/process/n16_003.csv',
 'data/20240213/process/n17_003.csv',
 'data/20240213/process/n18_001.csv',
 'data/20240213/process/n19_002.csv',
 ]

In [4]:
file_list = [i for i in file_list if i in report_file]
fname = [os.path.basename(i).replace('.csv','') for i in file_list]
comment_path = f'data/{day}/process/comment/'
comment_path

'data/20240213/process/comment/'

In [7]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, '2024 Next-Level Training Camp', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'{logo_path}/logo.png',
                   160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(0, 128, 0)
        self.set_text_color(255, 255, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'pelvis_rotational_velocity_with_respect_to_ground'  : ['Pelvic', 'r'],
    'trunk_rotational_velocity_with_respect_to_ground'   : ['Torso', 'g'],
    'lead_arm_rotational_velocity_with_respect_to_ground': ['Upper Arm','b'],
}

one_cols = {
    'trunk_twist_clockwise'                  : 'X FACTOR',
    'trunk_lateral_flexion'                  : 'TRUNK LATERAL TILT',
    'rear_shoulder_adduction'                : 'REAR SHOULDER AD-ABDUCTION',
    'lead_knee_extension'                    : 'LEAD KNEE FLEXION-EXTENSION',
    'lead_knee_extension_velocity'           : 'LEAD KNEE EXTENSION ANGULAR VELOCITY',
    'lead_hip_flexion_with_respect_to_trunk' : 'LEAD HIP FLEXION-EXTENSION',
    'shank_angle'                            : 'SHANK ANGLE',
    'hip_elbow_loss_space'                   : 'HIP-ELBOW DISTANCE',
    'shoulder_hand_loss_space'               : 'SHOULDER-HAND DISTANCE',
    'hand_shoulder_distance'                 : 'SHOULDER-HAND DISTANCE (AP)',
    'lead_elbow_flexion'                     : 'ELBOW FLEXION',
    'rear_shoulder_adduction_velocity'       : 'REAR SHOULDER AB-ADDUCTION VELOCITY',
    'lead_elbow_flexion_velocity'            : 'LEAD ELBOW EXTENSION VELOCITY',
    'pelvis_rotational_velocity_with_respect_to_ground' : 'PELVIS ROTATION VELOCITY'
}

lead_cols= {
    'lead_shoulder_external_rotation' : ['SHOULDER', 'darkblue'],
    'lead_elbow_flexion'              : ['ELBOW'   , 'firebrick'],
}

elbow_cols = {
    'rear_elbow_jc_3d_y' : ['REAR ELBOW', 'darkblue'],
    'lead_elbow_jc_3d_y' : ['LEAD ELBOW', 'firebrick'],
}

for file, name in tqdm(zip(file_list, fname)):
    df = pd.read_csv(file)
    to_time = df['to_frame'].values[0]
    kh_time = int(df['kh_frame'].values[0] - to_time)
    fc_time = int(df['fc_frame'].values[0] - to_time)
    bc_time = df['bc_frame'].values[0]
    pel_initial_time = int(df['pelvis_rotation_initial'][0] - to_time)
    df = df.iloc[int(to_time):int(bc_time+60)].reset_index(drop=True)
    
    bc_time = int(bc_time - to_time)
    total_time = int(bc_time - fc_time)
    to_time = 0
    time = df['time']
    
    store, _ = os.path.split(file)
    player = name.split('_')[0]
    trial = name.split('_')[1]
    weight = df['weight'][0]
    height = df['height'][0]
    side = df['handedness'][0]
    stride = round(df['stride_length'][0], 2)
    
    with open(f"{comment_path}/{name.split('_')[0]}_comment.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')
        
    position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE')
    position_xfactor = comment_txt.find('- X-FACTOR COMMENT')
    position_loading = comment_txt.find('- SWAY COMMENT')
    position_stride = comment_txt.find('- LOSS Of POSTURE COMMENT[Trunk Lateral Tilt]')
    position_daed_hands = comment_txt.find('- DEAD HANDS')
    position_casting_hand = comment_txt.find('- CASTING THE HANDS')
    position_loss_space_1 = comment_txt.find('- LOSS OF SPACE 1')
    position_loss_space_2 = comment_txt.find('- LOSS OF SPACE 2')
    position_loss_space_3 = comment_txt.find('- LOSS OF SPACE 3')
    position_rear_arm = comment_txt.find('- REAR SHOULDER AB/ADDUCTION ANGULAR VELOCITY')
    position_lead_knee_ext_vel = comment_txt.find('- LEAD LEG KNEE EXTENSION ANGULAR VELOCITY')
    position_lead_elb_ext_vel = comment_txt.find('- LEAD ELBOW EXTENSION ANGULAR VELOCITY')
    position_trunk = comment_txt.find('- TRUNK LATERAL TILT')
    
    comment_kinematic = comment_txt[position_kinematic:position_xfactor]
    comment_xfactor = comment_txt[position_xfactor:position_loading]
    comment_loading = comment_txt[position_loading:position_stride]
    comment_stride = comment_txt[position_stride:position_daed_hands]
    comment_dead_hands = comment_txt[position_daed_hands:position_casting_hand]
    comment_casting_hand = comment_txt[position_casting_hand:position_loss_space_1]
    comment_loss_space_1 = comment_txt[position_loss_space_1:position_loss_space_2]
    comment_loss_space_2 = comment_txt[position_loss_space_2:position_loss_space_3]
    comment_loss_space_3 = comment_txt[position_loss_space_3:position_rear_arm]
    comment_rear_arm = comment_txt[position_rear_arm:position_lead_knee_ext_vel]
    comment_lead_knee_ext_vel = comment_txt[position_lead_knee_ext_vel:position_lead_elb_ext_vel]
    comment_lead_elb_ext_vel = comment_txt[position_lead_elb_ext_vel:position_trunk]
    comment_trunk = comment_txt[position_trunk:]
    
    ks = report_data.kinematic_sequence(df,ks_cols, time, to_time, kh_time, fc_time, bc_time)
    ang = report_data.one_angle(df,one_cols, time, to_time, kh_time, fc_time, bc_time,pel_initial_time)
    arm = report_data.lead_arm(df,lead_cols, time, to_time, kh_time, fc_time, bc_time)
    elb = report_data.elbow_height(df,elbow_cols, time, to_time, kh_time, fc_time, bc_time)
    
    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)
    # ===========================================================================================================
    # main page
    pdf.add_page()
    # pdf.set_fill_color(r=150,g=0,b=0)
    # pdf.rect(w=210, h=297, x=0, y=0, style="DF")
    pdf.set_font('helvetica', 'BU', 20)
    title = '2024 Next-Level Training Camp'
    pdf.ln(10)
    title_w = pdf.get_string_width(title)
    doc_w = pdf.w
    # pdf.set_x(title_w/2)
    pdf.cell(0, 20, title, align = 'C', ln = True)

    # ===========================================================================================================
    # chapter
    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    ch12_link = pdf.add_link()
    ch13_link = pdf.add_link()
    ch14_link = pdf.add_link()
    ch15_link = pdf.add_link()
    ch16_link = pdf.add_link()
    ch17_link = pdf.add_link()
    ch18_link = pdf.add_link()
    
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0, 2,"",ln=1,align = 'C')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hitter Type : {side}   Weight : {weight}    Height : {height}", align = 'C', ln = True)
    pdf.cell(0, 5,"",ln=1,align = '')
    pdf.image(f'{logo_path}/analysis.png',x= 25,w=160,h=65)
    
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_fill_color(r=150,g=0,b=0)
    pdf.set_text_color(255,255,255)
    pdf.cell(0,7, 'SWING EFFICIENCY', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(100,5, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '2. PELVIC, TRUNK, ARM ANGULAR VELOCITY', ln = True, align = '', link = ch4_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '3. PEAK ANGULAR VELOCITY TIMING', ln = True, align = '', link = ch4_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '4. SPEED GAIN', ln = True, align = '', link = ch4_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '5. STRIDE LENGTH', ln = True, align = '', link = ch4_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '6. X-FACTOR', ln = True, align = '', link = ch4_link)
    pdf.cell(0,5,"",ln=1,align = '')
    
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_text_color(255,255,255)
    pdf.cell(90,7, 'LOADING PHASE', ln = False, align = 'C', fill=True,link = ch2_link)
    pdf.set_x(110)
    pdf.cell(90,7, 'STRIDE PHASE', ln = True, align = 'C', fill=True,link = ch4_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(100,5, '1. SWAY', ln = False, align = '', link = ch2_link)
    pdf.set_x(110)
    pdf.cell(0,5, '1. LOSS OF POSTURE', ln = True, align = '', link = ch4_link)
    pdf.cell(0,5,"",ln=1,align = '')

    pdf.set_font('helvetica', 'B', 15)
    pdf.set_text_color(255,255,255)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch6_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(0,5, '1. DEAD HANDS', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '2. CASTING THE HANDS', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '3. LOSS OF SPACE 1 [PELVIS OPEN TIMING]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '4. LOSS OF SPACE 2 [LEAD SHOULDER-HAND DISTANCE]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '5. LOSS OF SPACE 3 [ELBOW HEIGHT]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '6. REAR SHOULDER AB/ADDUCTION VELOCITY', ln = 1, align = '', link = ch15_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '7. LEAD LEG KNEE EXTENSION VELOCITY', ln = 1, align = '', link = ch16_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '8. LEAD ARM PEAK ELBOW EXTENSION VELOCITY', ln = 1, align = '', link = ch17_link)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.cell(0,5, '9. TRUNK TILT', ln = 1, align = '', link = ch18_link)
    
    # pdf.set_font('helvetica', 'B', 15)
    # pdf.cell(0,15,"",ln=1,align = '')
    # pdf.set_text_color(255,255,255)
    # pdf.cell(0,7, 'FEEDBACK', ln = 1, align = 'C',fill=True, link = ch11_link)
    # pdf.set_text_color(0,0,0)

    # ===============================================================================================
    peak_pel = ks['max']['pelvis_rotational_velocity_with_respect_to_ground']; time_pel = ks['time']['pelvis_rotational_velocity_with_respect_to_ground']
    peak_tor = ks['max']['trunk_rotational_velocity_with_respect_to_ground'];time_tor = ks['time']['trunk_rotational_velocity_with_respect_to_ground']
    peak_upper = ks['max']['lead_arm_rotational_velocity_with_respect_to_ground'];time_upper = ks['time']['lead_arm_rotational_velocity_with_respect_to_ground']
    # peak_fore = ks['max']['lead_shoulder_external_rotation_velocity'];time_fore = ks['time']['lead_shoulder_external_rotation_velocity']

    pel_time = round(100 * (time_pel - fc_time) / total_time, 2)
    tor_time = round(100 * (time_tor - fc_time) / total_time, 2)
    upper_time = round(100 * (time_upper - fc_time) / total_time, 2)
    # fore_time = round(100 * (time_fore - fc_time) / total_time, 2)
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_upper / peak_tor,2)
    # fore_gain = round(peak_fore / peak_upper,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING EFFICIENCY', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE', ln = True)
    pdf.image(f'figure/KINEMATIC SEQUENCE.png',x=75,y=15,w=133,h=50)
    pdf.image(f'{logo_path}/kinematic.png',x=17,y=15,w=50,h=50)
    pdf.set_font('helvetica', '', 9)
    pdf.set_x(35)
    pdf.set_y(65)
    pdf.set_text_color(150,0,0)
    pdf.cell(0,5, f"Stride Length : {stride} [% Height]", ln = 1, align = '')
    pdf.set_text_color(0,0,0)
    data_as_dict = {"Segment"                 : ["Pelvic"       , "Torso"        ,"Arm"            ],
                    "Pro Range"               : ["300 - 600"    , "600 - 800"    ,"850 - 2000"    ],
                    "Peak Velocity"           : [peak_pel       , peak_tor       ,peak_upper       ],
                    "Timing [BC - FC (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{upper_time} %"],
                    "Speed Gain"              : [''             , tor_gain       ,upper_gain       ]}
    
    pdf.set_y(70)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')
    
    pdf.set_font('helvetica', '', 9)
    pdf.set_y(58)
    pdf.set_text_color(150,0,0)
    pdf.cell(0,7,"",ln=1,align = '')
    pdf.cell(0,5, f"Pelvis Initial Rotation Time [BC - FC (100%)] : {round(100 * (pel_initial_time - fc_time) / total_time,2)} %", ln = 1, align = 'R')
    pdf.set_text_color(0,0,0)
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(100)
    # pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_kinematic, ln = True, border=False)
    
    # ===========================================================================================================
    # ===========================================================================================================
    pdf.set_link(ch2_link, page = 2)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'X FACTOR', ln = True)
    pdf.image('figure/X FACTOR.png',x=75,y=153,w=133,h=50)
    pdf.image(f'{logo_path}/xfactor.png',x=15,y=153,w=50,h=50)

    data_as_dict = {"":["Pro","Me"],
                    "Min"        : ['-26.12 ~ -16.14',ang['min']['trunk_twist_clockwise']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['trunk_twist_clockwise'] - fc_time) / total_time,2)} %"],
                    }
    
    pdf.set_y(205)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(228)
    pdf.multi_cell(0, 8, comment_xfactor, ln = True, border=False)
    
    # ===========================================================================================================
    # ===========================================================================================================
    # Shank Angle
    pdf.add_page()
    pdf.set_link(ch3_link, page = 3)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'LOADING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SWAY [SHANK ANGLE]', ln = True)
    pdf.image('figure/SHANK ANGLE.png',x=75,y=13,w=133,h=50)
    pdf.image(f'{logo_path}/shank_angle.png',x=15,y=13,w=50,h=50)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['1.83 ~ 13.23',ang['to_time']['shank_angle']],
                    "at Knee High"        : ['-',ang['kh_time']['shank_angle']],
                    "at Foot Contact"     : ['17.34 ~ 32.78',ang['fc_time']['shank_angle']],
                    "at Ball Contact"     : ['63.71 ~ 76.21',ang['bc_time']['shank_angle']]
                    }

    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Min"        : ['-0.75 ~ 8.95',ang['min']['shank_angle']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['shank_angle'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(85)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(108)
    pdf.multi_cell(0, 8, comment_loading, ln = True, border=False)
    # =============================================================================
    # Trunk Lataral Tilt [Loss of Posture]
    pdf.set_link(ch3_link, page = 3)
    pdf.set_text_color(255,255,255)
    pdf.set_y(145)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'STRIDE PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(155)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF POSTURE [TRUNK LATERAL TILT]', ln = True)
    pdf.image('figure/TRUNK LATERAL TILT.png',x=75,y=158,w=133,h=50)
    pdf.image(f'{logo_path}/lateral tilt.png',x=8,y=158,w=65,h=50)

    data_as_dict = {""                    : ["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-10.26 ~ 0.72', ang['to_time']['trunk_lateral_flexion']],
                    "at Knee High"        : ['-',ang['kh_time']['trunk_lateral_flexion']],
                    "at Foot Contact"     : ['-12.81 ~ -3.53',ang['fc_time']['trunk_lateral_flexion']],
                    "at Ball Contact"     : ['25.83 ~ 38.63',ang['bc_time']['trunk_lateral_flexion']]
                    }

    pdf.set_y(208)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" :["Pro", "Me"],
                    "Max"        : ['28.48 ~ 41.06',ang['max']['trunk_lateral_flexion']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['trunk_lateral_flexion'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(228)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(250)
    pdf.multi_cell(0, 8, comment_stride, ln = True, border=False)

    # ===================================================================================
    # LEAD SHOULDER - LEAD Hand DISTANCE 앞뒤 방향만 고려 [DEAD HANDS]
    pdf.add_page()
    pdf.set_link(ch4_link, page = 4)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'DEAD HANDS [LEAD SHOULDER - HAND DISTANCE]', ln = True)
    pdf.image(f'figure/SHOULDER-HAND DISTANCE (AP).png',x=75,y=15,w=133,h=50)
    pdf.image(f'{logo_path}/shoulder_hand_ap.png',x=17,y=12,w=50,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['-0 ~ -0.28',ang['to_time']['hand_shoulder_distance']],
                    "at Knee High"        : ['-',ang['kh_time']['hand_shoulder_distance']],
                    "at Foot Contact"     : ['-0.50 ~ -0.44',ang['fc_time']['hand_shoulder_distance']],
                    "at Ball Contact"     : ['0.40 ~ 0.46',ang['bc_time']['hand_shoulder_distance']]
                    }

    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro", "Me"],
                    "Min"        : ['-0.41 ~ -0.33 ',ang['min']['hand_shoulder_distance']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['hand_shoulder_distance'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(85)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')  
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(108)
    pdf.multi_cell(0, 8, comment_dead_hands, ln = True, border=False)   
    # ===========================================================================================================
    # Casting the Hand [Elbow Flexion]
    pdf.set_link(ch5_link, page = 4)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'CASTING THE HANDS [Elbow Flexion]', ln = True)
    pdf.image(f'figure/ELBOW FLEXION.png',x=75,y=153,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_flexion.png',x=17,y=153,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['lead_elbow_flexion']],
                    "at Knee High"        : [ang['kh_time']['lead_elbow_flexion']],
                    "at Foot Contact"     : [ang['fc_time']['lead_elbow_flexion']],
                    "at Ball Contact"     : [ang['bc_time']['lead_elbow_flexion']],
                    }
    pdf.set_y(205)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')
    
    data_as_dict = {
                    "at Max"            : [ang['max']['lead_elbow_flexion']],
                    "Max Timing"        : [f"{round(100 * (ang['max_time']['lead_elbow_flexion'] - fc_time) / total_time)} %"]
                    }
    pdf.set_y(220)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')    
    
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(237)
    pdf.multi_cell(0, 8, comment_casting_hand, ln = True, border=False)
      # ===================================================================================
    # PELVIS OPEN TIMING [Loss of Space 1]
    pdf.add_page()
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.set_link(ch6_link, page = 5)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 1 [PELVIS OPEN TIMING]', ln = True)
    pdf.image(f'figure/PELVIS ROTATION VELOCITY.png',x=75,y=15,w=133,h=50)
    pdf.image(f'{logo_path}/pel_rot_vel.png',x=17,y=15,w=50,h=50)

    data_as_dict = {"":["Pro", "Me"],
                    "Pelvic Open Timing"        : ["-40.32 ~ -4.26 %",f"{round(100 * (pel_initial_time - fc_time) / total_time,2)} %"],
                    "MAX"                       : ["300 - 600"       ,ang['max']['pelvis_rotational_velocity_with_respect_to_ground']],
                    "Max Timing"                : ["-"               ,f"{pel_time} %"],
                    }
    
    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Pelvis Open Timing (Loss of Space)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50,],
                    x_start='C')
    
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(95)
    pdf.multi_cell(0, 8, comment_loss_space_1, ln = True, border=False)  
     # ===================================================================================
    # LEAD SHOULDER - LEAD HAND 절대거리 [Loss of Space]
    pdf.set_link(ch7_link, page = 5)
    
    pdf.set_y(140)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 2 [LEAD SHOULDER - HAND DISTANCE]', ln = True)

    pdf.image(f'figure/SHOULDER-HAND DISTANCE.png',x=75,y=145,w=133,h=50)
    pdf.image(f'{logo_path}/shoulder_hand_loss.png',x=17,y=145,w=50,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['0.96 ~ 1.20',ang['to_time']['shoulder_hand_loss_space']],
                    "at Knee High"        : ['-',ang['kh_time']['shoulder_hand_loss_space']],
                    "at Foot Contact"     : ['0.84 ~ 1.08',ang['fc_time']['shoulder_hand_loss_space']],
                    "at Ball Contact"     : ['0.69 ~ 1.21',ang['bc_time']['shoulder_hand_loss_space']]
                    }

    pdf.set_y(195)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro", "Me"],
                    "Min"        : ['0.73 ~ 1.03',ang['min']['shoulder_hand_loss_space']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['shoulder_hand_loss_space'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(215)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')  
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(237)
    pdf.multi_cell(0, 8, comment_loss_space_2, ln = True, border=False)  
    # ===========================================================================================================
    # ELBOW HEIGHT
    pdf.add_page()
    pdf.set_link(ch8_link, page = 6)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 3 [ELBOW HEIGHT]', ln = True)

    pdf.image('figure/elbow_height.png',x=75,y=13,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_height.png',x=8,y=13,w=65,h=50)

    data_as_dict = {
                    "Segment"             : ["Rear Elbow"                        , "Lead Elbow"                       ,"My Displacement"    ,"Pro Displacement"                                             ],
                    "at Lead Leg Toe Off" : [elb['to_time']['rear_elbow_jc_3d_y'],elb['to_time']['lead_elbow_jc_3d_y'],round(elb['to_time']['rear_elbow_jc_3d_y']-elb['to_time']['lead_elbow_jc_3d_y'],2), '-0.12 ~ 0.22'],
                    "at Knee High"        : [elb['kh_time']['rear_elbow_jc_3d_y'],elb['kh_time']['lead_elbow_jc_3d_y'],round(elb['kh_time']['rear_elbow_jc_3d_y']-elb['kh_time']['lead_elbow_jc_3d_y'],2), '-'],
                    "at Foot Contact"     : [elb['fc_time']['rear_elbow_jc_3d_y'],elb['fc_time']['lead_elbow_jc_3d_y'],round(elb['fc_time']['rear_elbow_jc_3d_y']-elb['fc_time']['lead_elbow_jc_3d_y'],2), '0.04 ~ 0.26'],
                    "at Ball Contact"     : [elb['bc_time']['rear_elbow_jc_3d_y'],elb['bc_time']['lead_elbow_jc_3d_y'],round(elb['bc_time']['rear_elbow_jc_3d_y']-elb['bc_time']['lead_elbow_jc_3d_y'],2), '-0.40 ~ -0.32'],
                    }

    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')
    
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(100)
    pdf.multi_cell(0, 8, comment_loss_space_3, ln = True, border=False)
    # ===========================================================================================================
    # REAR SHOHULDER AD/ABDUCTION VELOCITY
    pdf.set_link(ch12_link, page = 6)
    pdf.set_y(140)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR SHOULDER AD/ABDUCTION ANGULAR VELOCITY', ln = True)

    pdf.image('figure/REAR SHOULDER AB-ADDUCTION VELOCITY.png',x=75,y=143,w=133,h=50)
    pdf.image(f'{logo_path}/add_vel.png',x=8,y=143,w=65,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['-141.67 ~ 80.61',ang['to_time']['rear_shoulder_adduction_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['rear_shoulder_adduction_velocity']],
                    "at Foot Contact"     : ['-212.5 ~ 158.1',ang['fc_time']['rear_shoulder_adduction_velocity']],
                    "at Ball Contact"     : ['-143.62 ~ 148.08',ang['bc_time']['rear_shoulder_adduction_velocity']]
                    }

    pdf.set_y(195)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" : ["Pro", "Me"] ,
                    "Max"        : ['157.28 ~ 527.46',ang['max']['rear_shoulder_adduction_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-", f"{round(100 * (ang['max_time']['rear_shoulder_adduction_velocity'] - fc_time) / total_time)} %"],
                    }
    pdf.set_y(215)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(237)
    pdf.multi_cell(0, 8, comment_rear_arm, ln = True, border=False)
    
    # ===========================================================================================================
    # Lead Knee Flexion Velcoity
    pdf.add_page()
    pdf.set_link(ch13_link, page = 7)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(10)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.image('figure/LEAD KNEE EXTENSION ANGULAR VELOCITY.png',x=75,y=13,w=133,h=50)
    pdf.image(f'{logo_path}/knee vel.png',x=8,y=13,w=65,h=55)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-171.41 ~ 30.37',ang['to_time']['lead_knee_extension_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['lead_knee_extension_velocity']],
                    "at Foot Contact"     : ['-136.51 ~ 53.51',ang['fc_time']['lead_knee_extension_velocity']],
                    "at Ball Contact"     : ['86.88 ~ 418.08',ang['bc_time']['lead_knee_extension_velocity']]
                    }

    pdf.set_y(65)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Max"        : ['239.09 ~ 983.27',ang['max']['lead_knee_extension_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['lead_knee_extension_velocity'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(85)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(107)
    pdf.multi_cell(0, 8, comment_lead_knee_ext_vel, ln = True, border=False)
    # ===========================================================================================================
    # Lead Elbow Extension Velcoity
    pdf.set_link(ch14_link, page = 7)
    
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD ELBOW EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.image('figure/LEAD ELBOW EXTENSION VELOCITY.png',x=75,y=153,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_vel.png',x=12,y=153,w=50,h=50)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-171.41 ~ 30.37',ang['to_time']['lead_elbow_flexion_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['lead_elbow_flexion_velocity']],
                    "at Foot Contact"     : ['-136.51 ~ 53.51',ang['fc_time']['lead_elbow_flexion_velocity']],
                    "at Ball Contact"     : ['86.88 ~ 418.08',ang['bc_time']['lead_elbow_flexion_velocity']]
                    }

    pdf.set_y(205)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Max"        : ['239.09 ~ 983.27',ang['max']['lead_elbow_flexion_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['lead_elbow_flexion_velocity'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(225)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(247)
    pdf.multi_cell(0, 8, comment_lead_elb_ext_vel, ln = True, border=False)
    # =============================================================================
    # Trunk Lataral Tilt
    pdf.add_page()
    pdf.set_link(ch15_link, page = 8)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK LATERAL TILT', ln = True)

    pdf.image('figure/TRUNK LATERAL TILT.png',x=75,y=20,w=133,h=50)
    pdf.image(f'{logo_path}/lateral tilt.png',x=8,y=20,w=65,h=50)

    data_as_dict = {""                    : ["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-10.26 ~ 0.72', ang['to_time']['trunk_lateral_flexion']],
                    "at Knee High"        : ['-',ang['kh_time']['trunk_lateral_flexion']],
                    "at Foot Contact"     : ['-12.81 ~ -3.53',ang['fc_time']['trunk_lateral_flexion']],
                    "at Ball Contact"     : ['25.83 ~ 38.63',ang['bc_time']['trunk_lateral_flexion']]
                    }

    pdf.set_y(70)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" :["Pro", "Me"],
                    "Max"        : ['28.48 ~ 41.06',ang['max']['trunk_lateral_flexion']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['trunk_lateral_flexion'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(120)
    pdf.multi_cell(0, 8, comment_trunk, ln = True, border='True')
    # =====================================================================================

    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)
    
    pdf.output(f'{store}/report/{day}_{player}_{trial}_baseball_report.pdf')

0it [00:00, ?it/s]

19it [03:04,  9.70s/it]
